In [1]:
class Location:
    def __init__(self, name, description, is_dark=False):
        self.name = name
        self.description = description
        self.connections = {}
        self.items = []
        self.is_locked = False
        self.unlock_item = None
        self.is_dark = is_dark

    def add_connection(self, direction, location, is_locked=False, unlock_item=None):
        self.connections[direction] = location
        if is_locked:
            self.is_locked = True
            self.unlock_item = unlock_item

    def add_item(self, item):
        self.items.append(item)

    def remove_item(self, item_name):
        self.items = [item for item in self.items if item.name.lower() != item_name.lower()]

    def get_description(self):
        description = f"You are at {self.name}. {self.description}\n"
        if self.items==[]:
          description+= "There are NO items here\n"
        else:
          description += "Items here: " + ', '.join([item.name for item in self.items]) + "\n"
        description += "Exits: " + ', '.join(self.connections.keys())
        return description


In [2]:

class Character:
    def __init__(self, name, description, location, is_hostile=False):
        self.name = name
        self.description = description
        self.location = location  # Reference to the location object where the character is
        self.is_hostile = is_hostile
        self.is_alive = True
        self.items = []  # Initialize items as an empty list

    def interact(self, player):
        """
        Interaction logic when the player meets this character.
        """
        if self.is_hostile:
            print(f"The {self.name} is hostile! It attacks you!")
            self.attack(player)
        else:
            print(f"You meet {self.name}. {self.description}")


    def defeat(self):

        if self.is_hostile:
            self.is_alive = False
            print(f"The {self.name} has been defeated.")
            # Drop the items the character was holding
            if self.items:
                print(f"The {self.name} dropped: {', '.join(item.name for item in self.items)}.")
                self.location.items.extend(self.items)  # Add dropped items to location






In [3]:
class Item:
    def __init__(self, name, description, is_collectable=True):
        self.name = name
        self.description = description
        self.is_collectable = is_collectable




In [4]:
class Player:
    def __init__(self, starting_location):
        self.location = starting_location
        self.inventory = []
        self.oxygen_level = 100
        self.oxygen_tank_used = False
        self.is_alive = True
        self.is_torchlight_on = False

    # Action-1
    def move(self, direction):
        if not self.is_alive:
            return

        # Check oxygen level before moving
        if self.oxygen_level <= 10:
            print("Your oxygen is running low! USE the Emergency Oxygen Tank to refill your oxygen cylinder.")
            if any(item.name.lower() == "emergency oxygen tank" for item in self.inventory):
                if not self.oxygen_tank_used:
                    print("You have Emergency oxygen cylinder in your inventory.(Refilling now!)")
                    self.oxygen_level = 100  # Refill oxygen
                    print("Oxygen Refill Successful!")
                    self.oxygen_tank_used = True  # Mark the tank as used
                else:
                    print("You already used the Emergency Oxygen Tank. No more refills available.")
                    self.oxygen_level = 0
                    print("You have suffocated! Game over.")
                    self.is_alive = False
                    return
            else:
                print("Your oxygen is critically low, and you don't have an Emergency Oxygen Tank!")
                print("You have suffocated! Game over.")
                self.is_alive = False
                return

        if direction in self.location.connections:
            next_location = self.location.connections[direction]

            # Move to the next location
            self.location = next_location
            self.oxygen_level -= 15
            print(f"You moved {direction} to {self.location.name}. Oxygen level is now {self.oxygen_level}%.")

            if next_location.is_locked:
                if any(item.name == next_location.unlock_item for item in self.inventory):
                    next_location.is_locked = False
                    print(f"You use the {next_location.unlock_item} to unlock the path.")
                else:
                    print(f"The path is locked. You need the {next_location.unlock_item} to proceed.")
                    return




            # If the new location is dark, notify the player
            if self.location.is_dark and not self.is_torchlight_on:
                print("It's too dark to see! You need to use the Torchlight to proceed.")

            return

        else:
            print("You can't go that way.")

    # Action-2
    def take(self, item_name):
        if not self.is_alive:
            return

        # Check if the room is dark before allowing the player to collect keys
        if self.location.is_dark and not self.is_torchlight_on:
            print("It's too dark to see anything. You need the Torchlight to find items here.")
            return

        # Special condition: Check if the player is trying to take the Vault Passcode and if the alien is defeated
        if item_name.lower() == "vault passcode":
          alien = next((character for character in game.characters if character.name == "Alien"), None)
          if alien and alien.is_alive:
            print("You cannot take the Vault Passcode! The Alien is still alive.")
            return

        if item_name.lower() == "artifact":
            # Check if the player has the Vault Passcode to take the Artifact
            if any(item_name.lower() == "vault passcode" for item in self.inventory):
                self.inventory.append(item)
                self.location.remove_item(item_name)
                print(f"You have collected the {item.name}.")
        item = next((item for item in self.location.items if item.name.lower() == item_name.lower()), None)
        if item and item.is_collectable:
            self.inventory.append(item)
            self.location.remove_item(item_name)
            print(f"You have collected the {item.name}.")
        else:
            print(f"There is no {item_name} here or it cannot be collected.")

    # Action-3
    def use_item(self, item_name):
        # If the item is the M-40 Gun and there is a hostile character (like Alien)
        if item_name in [item.name.lower() for item in self.inventory]:
          if item_name.lower() == "m-416 gun":
            hostile_characters = [character for character in game.characters if character.location == self.location and character.is_hostile and character.is_alive]
            if hostile_characters:
                for character in hostile_characters:
                    print(f"Firing the {item_name} ==> ==> ==> {character.name}.")
                    character.defeat()
            else:
                print("There's nothing hostile to attack here.")

          elif item_name.lower() == "torchlight" and self.location.is_dark:
            self.is_torchlight_on = True
            print("Switching on the Torchlight. The room is now illuminated.")
            return

        else:
          print(f"You don't have {item_name}.")
          return



    def inventory_list(self):
        return [item.name for item in self.inventory]

In [5]:
from graphviz import Digraph
class Game:
    def __init__(self):
        self.characters = []  # List to store all characters
        self.create_map()
        self.player = Player(self.landing_site)


    def create_map(self):
        # Locations
        self.landing_site = Location("Landing Site", "(The initial landing area with the spacecraft.)")
        self.supply_room = Location("Supply Room", "(A storage area with essential supplies.)")
        self.weapon_store = Location("Weapon Store", "(A store with Advanced weapons which are used in defeating Aliens.)")
        self.equipment_storage = Location("Equipment Storage", "A room with various equipment and tools.)", is_dark=True)  # Dark room
        self.base_station = Location("Base Station", "A secured base with restricted access. Alert: ALIEN inside. Defeat alien to get vault passcode.")
        self.martian_vault = Location("Martian Vault", "A vault where Mars artifacts are present.")

        # Characters
        self.alien = Character("Alien", "A strange creature with glowing eyes and sharp claws.", self.base_station, is_hostile=True)
        self.characters.append(self.alien)  # Add the alien to the game’s characters list
        # Connections
        self.landing_site.add_connection("south", self.supply_room)
        self.supply_room.add_connection("north", self.landing_site)
        self.supply_room.add_connection("west", self.weapon_store)
        self.weapon_store.add_connection("east", self.supply_room)
        self.supply_room.add_connection("east", self.equipment_storage)
        self.equipment_storage.add_connection("west", self.supply_room)
        self.supply_room.add_connection("south", self.base_station)
        self.base_station.add_connection("north", self.supply_room,is_locked=True, unlock_item="Keys to Base Station")
        self.base_station.add_connection("south", self.martian_vault)
        self.martian_vault.add_connection("north", self.base_station,is_locked=True, unlock_item="Vault Passcode")

        # Items
        self.supply_room.add_item(Item("Emergency Oxygen Tank", "A backup oxygen supply."))
        self.supply_room.add_item(Item("Torchlight", "A torch to see in the dark."))
        self.weapon_store.add_item(Item("M-416 Gun", "An advanced gun to kill alien."))
        self.weapon_store.add_item(Item("M-40 Gun", "A machine gun.")) #some other guns
        self.weapon_store.add_item(Item("AKM Gun", "Fast acting gun.")) #some other guns
        self.equipment_storage.add_item(Item("Keys to Base Station", "Keys to access the Base Station."))
        vault_passcode = Item("Vault Passcode", "A passcode for the Martian Vault security system.")
        self.alien.items.append(vault_passcode)  # Alien holds the Vault Passcode
        self.martian_vault.add_item(Item("Artifact", "Martian Artifact which contains valuable information about life on mars."))

    def generate_map_graph(self):
        dot = Digraph(comment='Mission to Mars Map')
        dot.attr(size='20,20')

        # Create nodes for each location with descriptions and items
        locations = [self.landing_site, self.supply_room, self.weapon_store, self.base_station, self.martian_vault, self.equipment_storage]
        for location in locations:
            item_list = ', '.join([item.name for item in location.items])
            label = f'{location.name}\n({location.description})\nItems: {item_list if item_list else "None"}'
            dot.node(location.name, label=label)

        # Create connections between locations
        for location in locations:
            for direction, connected_location in location.connections.items():
                dot.edge(location.name, connected_location.name, label=direction)

        # Render and save the graph to a file
        dot.render('mission_to_mars_map', format='png')
        print("Map generated and saved as 'mission_to_mars_map.png'.")

    # Updated game_loop to accept user commands
    def game_loop(self):
        print("Welcome to the Mission ~~ Mars!")

        while self.player.is_alive:

            print("*******************************************************\n")
            print(self.player.location.get_description())

            # Ask the user for a command
            command = input("Enter your command: ").lower().strip()

            if command.startswith("go "):
                direction = command.split(" ")[1]
                self.player.move(direction)

            elif command.startswith("take "):
                item_name = command.split("take ")[1]
                self.player.take(item_name)

            elif command == "inventory":
                print(f"You have: {', '.join(self.player.inventory_list())}")

            elif command.startswith("use "):
                item_name = command.split("use ")[1]
                self.player.use_item(item_name)

            elif command == "quit":
                print("Quitting game.")
                break

            else:
                print("Invalid command. Please try again.")

            # Win Condition: Check if player has collected the Artifact
            if self.player.location.name == "Martian Vault" and any(item.name.lower() == "artifact" for item in self.player.inventory):
              print("Congratulations! You've secured the Martian Artifact and completed the mission!\n")
              print("Mission Accomplished!\n")
              print("YOU WON!! Heading back to Home Earth!")
              self.player.is_alive = False
              break

            # Lose Condition: Check if the player is alive
            if not self.player.is_alive:
                print("You have died. Game over.")
                break



In [6]:
game = Game()
game.generate_map_graph()
game.game_loop()

Map generated and saved as 'mission_to_mars_map.png'.
Welcome to the Mission ~~ Mars!
*******************************************************

You are at Landing Site. (The initial landing area with the spacecraft.)
There are NO items here
Exits: south
Enter your command: go south
You moved south to Supply Room. Oxygen level is now 85%.
*******************************************************

You are at Supply Room. (A storage area with essential supplies.)
Items here: Emergency Oxygen Tank, Torchlight
Exits: north, west, east, south
Enter your command: take emergency oxygen tank
You have collected the Emergency Oxygen Tank.
*******************************************************

You are at Supply Room. (A storage area with essential supplies.)
Items here: Torchlight
Exits: north, west, east, south
Enter your command: take torchlight
You have collected the Torchlight.
*******************************************************

You are at Supply Room. (A storage area with essential suppli